# 네이버 블로그 크롤링
참고출처 : [네이버블로그크롤러](https://github.com/xotrs/naver-blog-crawler)
``` python
if __name__ == '__main__':
    naver_blog_crawling("파이썬 컨벤션", 100, "sim")
```
* first parameter : 검색하고 싶은 키워드
* second parameter : 한 페이지당 표출할 개수
* third parameter : 포스팅 정렬 순서(sim (유사도순) / date (날짜순))

In [2]:
# # 선행설치
# !pip install beautifulsoup4
# !pip install requests
# !pip install lxm
# # 연관분석
# !pip install apyori

ERROR: Could not find a version that satisfies the requirement lxm (from versions: none)
ERROR: No matching distribution found for lxm


In [3]:
import re
import json
import math
import datetime
import requests
import urllib.request
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup

In [ ]:
def naver_blog_crawling(search_blog_keyword, display_count, sort_type):
    search_result_blog_page_count = get_blog_search_result_pagination_count(search_blog_keyword, display_count)
    get_blog_post(search_blog_keyword, display_count, search_result_blog_page_count, sort_type

In [ ]:
def get_blog_search_result_pagination_count(search_blog_keyword, display_count):
    encode_search_keyword = urllib.parse.quote(search_blog_keyword)
    url = "https://openapi.naver.com/v1/search/blog?query=" + encode_search_keyword
    request = urllib.request.Request(url)

    request.add_header("X-Naver-Client-Id", naver_client_id)
    request.add_header("X-Naver-Client-Secret", naver_client_secret)

    response = urllib.request.urlopen(request)
    response_code = response.getcode()

    if response_code is 200:
        response_body = response.read()
        response_body_dict = json.loads(response_body.decode('utf-8'))

        if response_body_dict['total'] == 0:
            blog_pagination_count = 0
        else:
            blog_pagination_total_count = math.ceil(response_body_dict['total'] / int(display_count))
            #page 1000건 이상이면 1000건 까지 
            if blog_pagination_total_count >= 1000:
                blog_pagination_count = 1000
            else:
                blog_pagination_count = blog_pagination_total_count

            print("키워드 " + search_blog_keyword + "에 해당하는 포스팅 수 : " + str(response_body_dict['total']))
            print("키워드 " + search_blog_keyword + "에 해당하는 블로그 실제 페이징 수 : " + str(blog_pagination_total_count))
            print("키워드 " + search_blog_keyword + "에 해당하는 블로그 처리할 수 있는 페이징 수 : " + str(blog_pagination_count))

        return blog_pagination_count

In [75]:
def get_blog_post(search_blog_keyword, display_count, search_result_blog_page_count, sort_type):
    encode_search_blog_keyword = urllib.parse.quote(search_blog_keyword)
    # get_blog_search_result_pagination_count로 처리할 수있는 페이지수를 1부터 1000까지 각각 하나씩 긁어옴 
    for i in range(1, search_result_blog_page_count + 1):
        url = "https://openapi.naver.com/v1/search/blog?query=" + encode_search_blog_keyword + "&display=" + str(
            display_count) + "&start=" + str(i) + "&sort=" + sort_type

        request = urllib.request.Request(url)

        request.add_header("X-Naver-Client-Id", naver_client_id)
        request.add_header("X-Naver-Client-Secret", naver_client_secret)

        response = urllib.request.urlopen(request)
        response_code = response.getcode()
         #items의 개수만큼씩 진행
        if response_code is 200:
            response_body = response.read()
            response_body_dict = json.loads(response_body.decode('utf-8'))

            for j in range(0, len(response_body_dict['items'])):
                try:
                    blog_post_url = response_body_dict['items'][j]['link'].replace("amp;", "")

                    get_blog_post_content_code = requests.get(blog_post_url)
                    get_blog_post_content_text = get_blog_post_content_code.text

                    get_blog_post_content_soup = BeautifulSoup(get_blog_post_content_text, 'lxml')

                    for link in get_blog_post_content_soup.select('frame#mainFrame'):
                        real_blog_post_url = "http://blog.naver.com" + link.get('src')

                        get_real_blog_post_content_code = requests.get(real_blog_post_url)
                        get_real_blog_post_content_text = get_real_blog_post_content_code.text

                        get_real_blog_post_content_soup = BeautifulSoup(get_real_blog_post_content_text, 'lxml')

                        for blog_post_content in get_real_blog_post_content_soup.select('div#postViewArea'):
                            blog_post_content_text = blog_post_content.get_text()

                            remove_html_tag = re.compile('<.*?>')

                            blog_post_title = re.sub(remove_html_tag, '', response_body_dict['items'][j]['title'])
                            blog_post_description = re.sub(remove_html_tag, '',
                                                           response_body_dict['items'][j]['description'])
                            blog_post_postdate = datetime.datetime.strptime(response_body_dict['items'][j]['postdate'],
                                                                            "%Y%m%d").strftime("%y.%m.%d")
                            blog_post_blogger_name = response_body_dict['items'][j]['bloggername']
                            blog_post_full_contents = str(blog_post_content_text)

                            print("포스팅 URL : " + blog_post_url)
                            print("포스팅 제목 : " + blog_post_title)
                            print("포스팅 설명 : " + blog_post_description)
                            print("포스팅 날짜 : " + blog_post_postdate)
                            print("블로거 이름 : " + blog_post_blogger_name)
                            print("포스팅 내용 : " + blog_post_full_contents)
                except:
                    j += 1
    return blog_post_full_contents

In [173]:
response_body_dict['items'][j]['title']

'<b>양양</b> 횟집 싱싱함 한가득!'

In [26]:
#네이버 openAPI에서 발급받으면 사용가능
#비로그인방식으로 하루 25000건 사용가능 
naver_client_id = "cGDVdveeg4egAgRCvXtA"
naver_client_secret = "pFnYuTsat0"
search_blog_keyword = '양양'
sort_type = 'sim'

In [14]:
display_count =10

In [8]:
encode_search_keyword = urllib.parse.quote(search_blog_keyword)
url = "https://openapi.naver.com/v1/search/blog?query=" + encode_search_keyword
request = urllib.request.Request(url)

request.add_header("X-Naver-Client-Id", naver_client_id)
request.add_header("X-Naver-Client-Secret", naver_client_secret)

response = urllib.request.urlopen(request)
response_code = response.getcode()

In [10]:
if response_body_dict['total'] == 0:
            blog_pagination_count = 0
        else:
            blog_pagination_total_count = math.ceil(response_body_dict['total'] / int(display_count))
            #1000 page이상이면 1000가 최대 
            if blog_pagination_total_count >= 1000:
                blog_pagination_count = 1000
            else:
                blog_pagination_count = blog_pagination_total_count

In [16]:
blog_pagination_total_count

75472

In [23]:
encode_search_blog_keyword = urllib.parse.quote(search_blog_keyword)

In [27]:
for i in range(1, 2):
        url = "https://openapi.naver.com/v1/search/blog?query=" + encode_search_blog_keyword + "&display=" + str(
            display_count) + "&start=" + str(i) + "&sort=" + sort_type

In [29]:
request = urllib.request.Request(url)

request.add_header("X-Naver-Client-Id", naver_client_id)
request.add_header("X-Naver-Client-Secret", naver_client_secret)

response = urllib.request.urlopen(request)
response_code = response.getcode()

In [30]:
if response_code is 200:
    response_body = response.read()
    response_body_dict = json.loads(response_body.decode('utf-8'))

In [38]:
response_body_dict['items'][0]

{'title': '<b>양양</b>서핑 우와 뜨겁네!',
 'link': 'https://blog.naver.com/shakyamuni?Redirect=Log&logNo=221878085927',
 'description': '오늘은 <b>양양</b> 서핑 으로 동해에서 가장 핫한 서핑샵 플리즈웨잇서프를 소개해 드리려고 해요! <b>양양</b>은 강원도 내에서도 많이 유명한 지역이죠! 서퍼들이 많이 모이면서 동네 자체가 아주 힙하고 독특한... ',
 'bloggername': '피위의 행복한 어드벤쳐',
 'bloggerlink': 'https://blog.naver.com/shakyamuni',
 'postdate': '20200328'}

In [161]:
#items의 개수만큼씩 진행
postdates = []
strings =[]
for j in range(0, len(response_body_dict['items'])):
    blog_post_url = response_body_dict['items'][j]['link'].replace("amp;", "")

    get_blog_post_content_code = requests.get(blog_post_url)
    get_blog_post_content_text = get_blog_post_content_code.text

    get_blog_post_content_soup = BeautifulSoup(get_blog_post_content_text, 'lxml')
    #print(get_blog_post_content_soup.select('#mainFrame'))
    for link in get_blog_post_content_soup.select('#mainFrame'):
        real_blog_post_url = "http://blog.naver.com" + link.get('src')

        get_real_blog_post_content_code = requests.get(real_blog_post_url)
        get_real_blog_post_content_text = get_real_blog_post_content_code.text

        get_real_blog_post_content_soup = BeautifulSoup(get_real_blog_post_content_text, 'lxml')

        for blog_post_content in get_real_blog_post_content_soup.select('div#postViewArea'):
            
            #전체 텍스트 
            string = ""
            for sentence in blog_post_content.stripped_strings:
                string += " "+sentence.replace('\xa0'," ")
            strings.append(string) 
   
            #포스트날짜
            postdate = datetime.datetime.strptime(response_body_dict['items'][j]['postdate'],"%Y%m%d").strftime("%y.%m.%d")
            postdates.append(postdate)

In [172]:
blog_post_content

<div id="postViewArea">
<div class="post-view pcol2 _param(1) _postViewArea221841451893" id="post-view221841451893">
<div align="center"><span style="">얼마전에 가족들과 양양에 놀러갔다가<br/>아주 괜찮은 양양 맛집을 찿아갔어요~<br/>대게 전문점이었는데 신선함은 물론이고<br/>푸짐하고 든든한 한끼를 제대로 먹고 올 수있어서<br/>만족했던 곳이에요<br/></span></div><div align="left"><img alt="" class="m20 _attach_space" src="https://ssl.pstatic.net/static/blank.gif"/></div><div align="center"><img alt="" class="_photoImage" id="MjAyMDAzMDdfMTM2/MDAxNTgzNTI5MjY2MzQ2.sQKXX5yFXE5RpRhIT_eGCE0EsBloL5PpoJ3arTXlt-kg.udvxPWuy4d8Yyw0iDeZdCGoEMwLa_OSfNEVkNmmD94sg.JPEG.collier13/NaverBlog_20200307_061425_01.jpg" onclick="popview(this, '90000003_0000000000000033A6C7EB75')" onload="setTimeout('utility.resizeImageWithId(\'MjAyMDAzMDdfMTM2/MDAxNTgzNTI5MjY2MzQ2.sQKXX5yFXE5RpRhIT_eGCE0EsBloL5PpoJ3arTXlt-kg.udvxPWuy4d8Yyw0iDeZdCGoEMwLa_OSfNEVkNmmD94sg.JPEG.collier13/NaverBlog_20200307_061425_01.jpg\')',200)" src="http://postfiles1.naver.net/MjAyMDAzMDdfMTM2/MDAxNTgzNTI5MjY2MzQ2.sQKXX5y

In [169]:
real_blog_post_url

'http://blog.naver.com/PostView.nhn?blogId=rlawngmls2&logNo=221818034398&from=search&redirect=Log&widgetTypeCall=true&directAccess=false'

In [168]:
get_blog_post_content_soup

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="ko">
<head>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="-1" http-equiv="Expires"/>
<meta content="noindex,follow" name="robots"/>
<meta content="always" name="referrer"/>
<meta content="text/html;charset=utf-8" http-equiv="content-type"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<link href="/favicon.ico?2" rel="shortcut icon" type="image/x-icon"/>
<link href="https://rss.blog.naver.com/rlawngmls2.xml" rel="alternate" title="RSS feed for rlawngmls2 Blog" type="application/rss+xml"/>
<link href="https://blog.naver.com/NBlogWlwLayout.nhn?blogId=rlawngmls2" rel="wlwmanifest" type="application/wlwmanifest+xml"/>
<title>끼임주히블로그 : 네이버 블로그</title>
</head>
<script charset="UTF-8" src="https://ssl.pstatic.net/t.static.blog/mylog/versioning/Frameset-657805386_https.js" type="text/javascript"></script>
<script charset="UTF-8"

In [164]:
postdates

['20.03.07']

In [153]:
strings

[]

In [154]:
string = ""
for sentence in blog_post_content[0].stripped_strings:
    string += " "+sentence.replace('\xa0'," ")
strings.append(string) 

In [70]:
real_blog_post_url

'http://blog.naver.com/PostView.nhn?blogId=collier13&logNo=221841451893&from=search&redirect=Log&widgetTypeCall=true&directAccess=false'

In [33]:
response_body_dict['items']

10

In [93]:
get_real_blog_post_content_soup = BeautifulSoup(get_real_blog_post_content_text, 'html.parser')

In [96]:
blog_post_content = get_real_blog_post_content_soup.select('div#postViewArea')

In [171]:
blog_post_content

<div id="postViewArea">
<div class="post-view pcol2 _param(1) _postViewArea221841451893" id="post-view221841451893">
<div align="center"><span style="">얼마전에 가족들과 양양에 놀러갔다가<br/>아주 괜찮은 양양 맛집을 찿아갔어요~<br/>대게 전문점이었는데 신선함은 물론이고<br/>푸짐하고 든든한 한끼를 제대로 먹고 올 수있어서<br/>만족했던 곳이에요<br/></span></div><div align="left"><img alt="" class="m20 _attach_space" src="https://ssl.pstatic.net/static/blank.gif"/></div><div align="center"><img alt="" class="_photoImage" id="MjAyMDAzMDdfMTM2/MDAxNTgzNTI5MjY2MzQ2.sQKXX5yFXE5RpRhIT_eGCE0EsBloL5PpoJ3arTXlt-kg.udvxPWuy4d8Yyw0iDeZdCGoEMwLa_OSfNEVkNmmD94sg.JPEG.collier13/NaverBlog_20200307_061425_01.jpg" onclick="popview(this, '90000003_0000000000000033A6C7EB75')" onload="setTimeout('utility.resizeImageWithId(\'MjAyMDAzMDdfMTM2/MDAxNTgzNTI5MjY2MzQ2.sQKXX5yFXE5RpRhIT_eGCE0EsBloL5PpoJ3arTXlt-kg.udvxPWuy4d8Yyw0iDeZdCGoEMwLa_OSfNEVkNmmD94sg.JPEG.collier13/NaverBlog_20200307_061425_01.jpg\')',200)" src="http://postfiles1.naver.net/MjAyMDAzMDdfMTM2/MDAxNTgzNTI5MjY2MzQ2.sQKXX5y

In [170]:
get_real_blog_post_content_soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html lang="ko">
<head>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="-1" http-equiv="Expires"/>
<meta content="always" name="referrer"/>
<!--[if ie]>
<style type="text/css">
html {overflow: scroll; overflow-x: auto;}
</style>
<![endif]-->
<link charset="UTF-8" href="http://t.static.blog.naver.net/mylog/versioning/LayoutTopCommon-598625523.css" rel="stylesheet" type="text/css"/>
<link href="http://t.static.blog.naver.net/mylog/versioning//common/css/music/player-d3fc09e.css" rel="stylesheet" type="text/css"/>
<link href="/favicon.ico?2" rel="shortcut icon" type="image/x-icon"/>
<meta content="양양 횟집 싱싱함 한가득!" property="og:title"/>
<meta content="http://blogthumb2.naver.net/MjAyMDAyMTlfMTY1/MDAxNTgyMTExMjM2NTI1.VqbY7t4Y81CNH4cXP_iV_ZmLGN5L91o3BMSEr3E2R6gg.JLE8Jy4FHBtg7gEE22pjD6j3MsJYim4hezgMgrUzXUsg.JPEG.rlawngmls2/1582111235839.JPG?type=w2" property="og:image"/>
<meta c

In [111]:
blog_post_content[0].strings

TypeError: 'generator' object is not callable

In [141]:
text = ''

In [142]:
for string in blog_post_content[0].stripped_strings:
    a += " "+string.replace('\xa0'," ")

In [144]:
blog_post_postdate

'20.03.07'

In [ ]:
def get_blog_post(keyword, display_count, page_count, sort_type):
    #날짜와 본문내용 저장
    postdates = []
    strings =[]
    
    encode_keyword = urllib.parse.quote(keyword)
    # get_blog_search_result_pagination_count로 처리할 수있는 페이지수를 1부터 페이지수까지 까지 각각 하나씩 긁어옴 
    for i in tqdm_notebook(range(1, page_count + 1),desc = "page work"):
        url = "https://openapi.naver.com/v1/search/blog?query=" + encode_keyword + "&display=" + str(
            display_count) + "&start=" + str(i) + "&sort=" + sort_type

        request = urllib.request.Request(url)

        request.add_header("X-Naver-Client-Id", naver_client_id)
        request.add_header("X-Naver-Client-Secret", naver_client_secret)

        response = urllib.request.urlopen(request)
        response_code = response.getcode()
         
        if response_code is 200:
            response_body = response.read()
            response_body_dict = json.loads(response_body.decode('utf-8'))
        #items의 개수만큼씩 진행
            for j in range(0, len(response_body_dict['items'])):
                try:
                    blog_post_url = response_body_dict['items'][j]['link'].replace("amp;", "")
                    #블로그 url안에 들어가기(아직 크롤링불가)
                    get_blog_post_content_code = requests.get(blog_post_url)
                    get_blog_post_content_text = get_blog_post_content_code.text

                    get_blog_post_content_soup = BeautifulSoup(get_blog_post_content_text, 'lxml')
                    #크롤링가능한 url에 접속
                    for link in get_blog_post_content_soup.select('frame#mainFrame'):
                        real_blog_post_url = "http://blog.naver.com" + link.get('src')

                        get_real_blog_post_content_code = requests.get(real_blog_post_url)
                        get_real_blog_post_content_text = get_real_blog_post_content_code.text
                        get_real_blog_post_content_soup = BeautifulSoup(get_real_blog_post_content_text, 'lxml')
                        #본문부분 추출 
                        for blog_post_content in get_real_blog_post_content_soup.select('div#postViewArea'):
                            blog_post_content_text = blog_post_content.get_text()

                            #전체 텍스트 
                            string = ""
                            for sentence in blog_post_content.stripped_strings:
                                string += " "+sentence.replace('\xa0'," ")
                            strings.append(string) 

                            #포스트날짜
                            postdate = datetime.datetime.strptime(response_body_dict['items'][j]['postdate'],"%Y%m%d").strftime("%y.%m.%d")
                            postdates.append(postdate)
                except:
                    j += 1
    return postdates, strings